## 01. Introduction to Graph(연산/계산 설계도)

### 1-1. Default Graph 사용
- 연산을 위한 설계도(Graph)를 별도 정의하지 않으면, 기본적으로 제공되는 Graph가 자동 적용.

In [1]:
import tensorflow as tf

print(tf.__version__)

D:\anaconda3\envs\tf16\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\tf16\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\tf16\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\tf16\lib\site-packages\tensorflow\python\framework\dtypes.py:496: FutureWarning: Passing (type

1.5.1


In [2]:
a1 = tf.Variable(3)
b1 = tf.Variable(5)
c1 = a1 + b1

sess1 = tf.Session()       #.Session() 괄호에 아무것도 없으면 기본 Graph가 자동 적용됨.
sess1.run(tf.global_variables_initializer())
print(sess1.run(c1))       # 계산 결과는 print안에서 .run을 호출해야 함.

8


### Session 저장

In [3]:
saver = tf.train.Saver()
saver.save(sess1,'saved/my_test_sess1')

'saved/my_test_sess1'

### 1-2. Customer Graph 사용
- epoch step 별 session 저장 가능

In [4]:
cg1 = tf.Graph()
sess2 = tf.Session(graph = cg1)


with cg1.as_default():

    a2 = tf.Variable(3)
    b2 = tf.Variable(5)
    c2 = a2 + b2
    
    sess2.run(tf.global_variables_initializer())
    
    #모델 저장 시 별도의 saver를 만들어준다.
    saver2 = tf.train.Saver()
    
    for step in range(3):
        ckpt_path = saver2.save(sess2,'saved/my_test_sess2', step)
        
        print("ckpt file:", ckpt_path)
    
    writer = tf.summary.FileWriter("./log", sess2.graph)
    
    #summary_writer = tf.train.SummaryWriter("my_sess2_summary", sess2.graph)
    
print(sess2.run(c2))

ckpt file: saved/my_test_sess2-0
ckpt file: saved/my_test_sess2-1
ckpt file: saved/my_test_sess2-2
8


In [5]:
# tensorboard 확인은 conda prompt window에서 입력
tensorboard --logdir=./log

SyntaxError: invalid syntax (<ipython-input-5-37a18c54f86a>, line 2)

### 1-3. session 실행이 맞지 않으면 Error 발생

In [6]:
cg2 = tf.Graph()

with cg2.as_default():
    a3 = tf.Variable(3)
    b3 = tf.Variable(5)
    c3 = a3 + b3
    
sess3 = tf.Session(graph = cg2)              # with 구문 밖으로 나왔음.
sess3.run(tf.global_variables_initializer())
print(sess2.run(c3))                         # sess2에 c3을 연산하므로 맞지 않음

ValueError: Fetch argument <tf.Operation 'init_1' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init_1"
op: "NoOp"
input: "^Variable/Assign"
input: "^Variable_1/Assign"
 is not an element of this graph.)

### 1-4. Graph와 Session을 따로 정의하는 방법

In [7]:
cg3 = tf.Graph()

with cg3.as_default():
    v0 = tf.placeholder(tf.int32, name = "V0")
    v1 = tf.Variable(10, name = "V1")
    v2 = tf.Variable(20, name = "V2")
    v3 = tf.add(v0,v2, name = "add")
    
with tf.Session(graph = cg3) as sess3:
    saver3 = tf.train.Saver()
    sess3.run(tf.global_variables_initializer())
    save_path = saver3.save(sess3, "./saved/my_test_sess3")
    feed_dict = {v0:7}
    output = sess3.run([v3], feed_dict = feed_dict)
    
    writer = tf.summary.FileWriter("./log", sess3.graph)

print(output)

[27]


### 1-5. graph 소속 여부 확인

In [8]:
c2.graph is tf.get_default_graph()

False

In [9]:
c2.graph is cg1

True

### 1-6. 사용 중인 graph node 확인

In [10]:
[node.name for node in cg3.as_graph_def().node]            # cg3.get_operations() 와 동일 결과

['V0',
 'V1/initial_value',
 'V1',
 'V1/Assign',
 'V1/read',
 'V2/initial_value',
 'V2',
 'V2/Assign',
 'V2/read',
 'add',
 'save/Const',
 'save/SaveV2/tensor_names',
 'save/SaveV2/shape_and_slices',
 'save/SaveV2',
 'save/control_dependency',
 'save/RestoreV2/tensor_names',
 'save/RestoreV2/shape_and_slices',
 'save/RestoreV2',
 'save/Assign',
 'save/RestoreV2_1/tensor_names',
 'save/RestoreV2_1/shape_and_slices',
 'save/RestoreV2_1',
 'save/Assign_1',
 'save/restore_all',
 'init']

In [11]:
cg3.get_operations()

[<tf.Operation 'V0' type=Placeholder>,
 <tf.Operation 'V1/initial_value' type=Const>,
 <tf.Operation 'V1' type=VariableV2>,
 <tf.Operation 'V1/Assign' type=Assign>,
 <tf.Operation 'V1/read' type=Identity>,
 <tf.Operation 'V2/initial_value' type=Const>,
 <tf.Operation 'V2' type=VariableV2>,
 <tf.Operation 'V2/Assign' type=Assign>,
 <tf.Operation 'V2/read' type=Identity>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'save/Const' type=Const>,
 <tf.Operation 'save/SaveV2/tensor_names' type=Const>,
 <tf.Operation 'save/SaveV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/SaveV2' type=SaveV2>,
 <tf.Operation 'save/control_dependency' type=Identity>,
 <tf.Operation 'save/RestoreV2/tensor_names' type=Const>,
 <tf.Operation 'save/RestoreV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/RestoreV2' type=RestoreV2>,
 <tf.Operation 'save/Assign' type=Assign>,
 <tf.Operation 'save/RestoreV2_1/tensor_names' type=Const>,
 <tf.Operation 'save/RestoreV2_1/shape_and_slices' type=Const>,
 <tf

In [12]:
[node.input for node in cg3.as_graph_def().node]       # .input, op, device, attr 등 확인 가능

[[],
 [],
 [],
 ['V1', 'V1/initial_value'],
 ['V1'],
 [],
 [],
 ['V2', 'V2/initial_value'],
 ['V2'],
 ['V0', 'V2/read'],
 [],
 [],
 [],
 ['save/Const', 'save/SaveV2/tensor_names', 'save/SaveV2/shape_and_slices', 'V1', 'V2'],
 ['save/Const', '^save/SaveV2'],
 [],
 [],
 ['save/Const', 'save/RestoreV2/tensor_names', 'save/RestoreV2/shape_and_slices'],
 ['V1', 'save/RestoreV2'],
 [],
 [],
 ['save/Const', 'save/RestoreV2_1/tensor_names', 'save/RestoreV2_1/shape_and_slices'],
 ['V2', 'save/RestoreV2_1'],
 ['^save/Assign', '^save/Assign_1'],
 ['^V1/Assign', '^V2/Assign']]

- 참고: https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/tool_developers/

### 1-7. graph 요소 확인

In [13]:
graph_collection_key_list = cg1.get_all_collection_keys()
graph_collection_list = cg1.get_collection(graph_collection_key_list[0])
oper_list = cg1.get_operations()            # cg1.as_graph_def().node와 동일 결과

In [14]:
graph_collection_key_list

['trainable_variables', 'variables']

In [15]:
graph_collection_list

[<tf.Variable 'Variable:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref>]

In [16]:
oper_list

[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'Variable_1/initial_value' type=Const>,
 <tf.Operation 'Variable_1' type=VariableV2>,
 <tf.Operation 'Variable_1/Assign' type=Assign>,
 <tf.Operation 'Variable_1/read' type=Identity>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'save/Const' type=Const>,
 <tf.Operation 'save/SaveV2/tensor_names' type=Const>,
 <tf.Operation 'save/SaveV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/SaveV2' type=SaveV2>,
 <tf.Operation 'save/control_dependency' type=Identity>,
 <tf.Operation 'save/RestoreV2/tensor_names' type=Const>,
 <tf.Operation 'save/RestoreV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/RestoreV2' type=RestoreV2>,
 <tf.Operation 'save/Assign' type=Assign>,
 <tf.Operation 'save/RestoreV2_1/tensor_names' type=Const>,
 <tf.Operation 's

### 1-8. 저장된 model file을 불러와서 Graph node 확인

- 불러온 모델의 graph 저장 방법 1

In [17]:
im1_graph = tf.Graph()
with im1_graph.as_default():
    im1_saver = tf.train.import_meta_graph("./saved/my_test_sess3.meta")
    train_op = tf.get_collection('variables')         # variables 확인

print(train_op)
[node.name for node in im1_graph.as_graph_def().node] 

[<tf.Variable 'V1:0' shape=() dtype=int32_ref>, <tf.Variable 'V2:0' shape=() dtype=int32_ref>]


['V0',
 'V1/initial_value',
 'V1',
 'V1/Assign',
 'V1/read',
 'V2/initial_value',
 'V2',
 'V2/Assign',
 'V2/read',
 'add',
 'save/Const',
 'save/SaveV2/tensor_names',
 'save/SaveV2/shape_and_slices',
 'save/SaveV2',
 'save/control_dependency',
 'save/RestoreV2/tensor_names',
 'save/RestoreV2/shape_and_slices',
 'save/RestoreV2',
 'save/Assign',
 'save/RestoreV2_1/tensor_names',
 'save/RestoreV2_1/shape_and_slices',
 'save/RestoreV2_1',
 'save/Assign_1',
 'save/restore_all',
 'init']

In [18]:
with tf.Session(graph = im1_graph) as im1_sess:
    im1_saver.restore(im1_sess, tf.train.latest_checkpoint('./saved'))

    # 기존 graph에 있는 변수를 가져와 새로 지정
    new_v0 = im1_graph.get_tensor_by_name("V0:0")
    new_v3 = im1_graph.get_tensor_by_name("add:0")
    feed_dict = {new_v0:7}
    output1 = im1_sess.run([new_v3], feed_dict = feed_dict)
    
    # 기존 그래프에 새로운 그래프를 연결하여 연산
    new_v1 = im1_graph.get_tensor_by_name("V1:0")
    new_op = tf.multiply(new_v1, 3, name="multiply")
    output2 = im1_sess.run(new_op)    
    
        
print(output1)
print(output2)

INFO:tensorflow:Restoring parameters from ./saved\my_test_sess3
[27]
30


## 1-9. pb file 만들기 (Freeze)

- 그래프를 고정한다 라고 이야기 하기도 함.
- 더 이상 학습이 안되도록 모델의 구조를 가중치와 결합하는 작업.
- 안드로이드나 다른 디바이스에 포팅하기 위함

** tf.train.write_graph( graph_or_graph_def, logdir, name, as_text=True )
- as_text=True 또는 생략은 pbtxt 파일이 생성되며, False는 pb파일이 생성됨.

In [19]:
tf.train.write_graph(sess3.graph_def,'./saved','my_test_sess3.pbtxt', as_text=True)
tf.train.write_graph(sess3.graph_def,'./saved','my_test_sess3.pb', as_text=False)

'./saved\\my_test_sess3.pb'

## 1-10. pb file 불러와서 graph에 입력하기

In [20]:
im2_graph = tf.Graph()
with im2_graph.as_default():
    im2_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile('./saved/my_test_sess3.pb', 'rb') as f:
        im2_graph_def.ParseFromString(f.read())
        tf.import_graph_def(im2_graph_def, name="")

In [21]:
[node.name for node in im2_graph.as_graph_def().node] 

['V0',
 'V1/initial_value',
 'V1',
 'V1/Assign',
 'V1/read',
 'V2/initial_value',
 'V2',
 'V2/Assign',
 'V2/read',
 'add',
 'save/Const',
 'save/SaveV2/tensor_names',
 'save/SaveV2/shape_and_slices',
 'save/SaveV2',
 'save/control_dependency',
 'save/RestoreV2/tensor_names',
 'save/RestoreV2/shape_and_slices',
 'save/RestoreV2',
 'save/Assign',
 'save/RestoreV2_1/tensor_names',
 'save/RestoreV2_1/shape_and_slices',
 'save/RestoreV2_1',
 'save/Assign_1',
 'save/restore_all',
 'init']

In [22]:
# tf.initialize_all_variables().run()

with tf.Session(graph=im2_graph) as im2_sess:
    # 기존 graph에 있는 변수를 가져와 새로 지정
    im2_sess.run(tf.global_variables_initializer())
    new_v0 = im2_graph.get_tensor_by_name("V0:0")
    new_v3 = im2_graph.get_tensor_by_name("add:0")
    feed_dict = {new_v0:7}
    output1 = im2_sess.run([new_v3], feed_dict = feed_dict)

print(output1)

FailedPreconditionError: Attempting to use uninitialized value V2
	 [[Node: V2/read = Identity[T=DT_INT32, _class=["loc:@V2"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](V2)]]

Caused by op 'V2/read', defined at:
  File "D:\anaconda3\envs\tf16\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda3\envs\tf16\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda3\envs\tf16\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anaconda3\envs\tf16\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\anaconda3\envs\tf16\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "D:\anaconda3\envs\tf16\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "D:\anaconda3\envs\tf16\lib\asyncio\base_events.py", line 442,